In [1]:
from os.path import join
import glob
import matplotlib.pyplot as plt
import pandas as pd
from transformers import BertModel, BertTokenizer, XLNetModel, XLNetTokenizer
import tensorflow as tf
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data as data_utils
import numpy as np
from typing import List
from tqdm import tqdm
import pickle
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import itertools
from tensorflow.keras import layers, models
import seaborn as sn
tqdm.pandas()
# drive.mount('/content/drive')


## Data loading

In [4]:
data = pd.read_csv(r"C:\Users\dupho\1_quinn/1Thesis/main_dataset/01trainingdata/raw_file.csv", index_col = 'id')

In [5]:
data.head()

,label,detail_label,text
id,,,
0,行销 活动 问题,未 收到 活动 回馈 金 问题,2020 7 6 午 5 38 左右 绑定 兆丰 银行 开 启动 储值功...
1,行销 活动 问题,未 收到 活动 点数 问题,刷 屈臣氏 480 元数 回馈
2,行销 活动 问题,未 收到 活动 点数 问题,确认 46 喔 5 数漏 发 5 帐单 缴费 13075 元 应道 13075 * ...
3,行销 活动 问题,未 收到 活动 点数 问题,问外 手机号码 卡通 缴 电费 没 回馈 谢谢
4,行销 活动 问题,未 收到 活动 点数 问题,hi 意 行销 条款 勾 绿色 勾勾 2 姓名 林 * 炜 身份证 f12 ...


In [11]:
len(data.label.value_counts()) # 11 labels

11

In [10]:
len(data.detail_label.value_counts()) # 11 labels

47

In [12]:
data.dropna(inplace=True)

In [13]:
data.shape

(6947, 3)

## Embedding

In [14]:
model = BertModel.from_pretrained('bert-base-chinese') # bert-chinese
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

#### Model test

In [15]:
encoded_input = tokenizer.encode(data.iloc[2]['text'], return_tensors='pt')
outputs = model(encoded_input)

In [16]:
encoded_input.shape

torch.Size([1, 39])

In [17]:
len(data.iloc[2763]['text'].split())

31

In [18]:
# output of each layer
outputs[1].shape

torch.Size([1, 768])

In [19]:
# output of the embeddings
len(outputs[1].resize(outputs[1].shape[1]))

C:\Users\dupho\anaconda3\envs\pytorch\lib\site-packages\torch\_tensor.py:490: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


768

#### Data embedding

In [20]:
# class FrameDataset(data_utils.Dataset):
#   def __init__(
#       self,
#       dataFrame: pd.DataFrame,
#       x_label: str,
#       y_label: str
#   ):
#     self.df = dataFrame
#     self.x = list(self.df[x_label])
#     self.y = list(self.df[y_label])

#   def __getitem__(self, idx):
#     return self.x[idx], self.y[idx]

#   def __len__(self):
#     return len(self.y)

class Embedder(nn.Module):
  def __init__(self, tokenizer, embedder):
    super(Embedder, self).__init__()
    self.tokenizer = tokenizer
    self.embedder = model

  def forward(self, x):
    if type(x) == str and len(x) > 0:
      with torch.no_grad():
        x = self.tokenizer(x, padding=True, truncation=True, max_length=128, return_tensors='pt')
        x = self.embedder(**x)[1]
        x = x.numpy()
        x = x.reshape(x.shape[1])
    else:
      x = np.zeros(768)
    return x

In [21]:
# data = FrameDataset(df, 'tweet', 'emotion')
# dataloader = data_utils.DataLoader(data, batch_size=100, shuffle=False)

embedder = Embedder(tokenizer, model)

In [22]:
data['text_vec'] = data.text.progress_apply(embedder)

100%|██████████| 6947/6947 [07:48<00:00, 14.84it/s]


In [ ]:
# embeddings = []
# for i, data in tqdm(enumerate(dataloader, 0)):

#     inputs, labels = data
#     outputs = embedder(inputs)
#     embeddings.append(outputs.numpy())

In [23]:
data.to_csv(r"C:\Users\dupho\1_quinn/1Thesis/main_dataset/01trainingdata/raw_file_vec.csv")

In [24]:
data.head()

,label,detail_label,text,text_vec
id,,,,
0,行销 活动 问题,未 收到 活动 回馈 金 问题,2020 7 6 午 5 38 左右 绑定 兆丰 银行 开 启动 储值功...,"[0.9987324, 0.99997675, 0.98749787, 0.98941666..."
1,行销 活动 问题,未 收到 活动 点数 问题,刷 屈臣氏 480 元数 回馈,"[0.9998949, 0.9999919, 0.99948376, 0.9852125, ..."
2,行销 活动 问题,未 收到 活动 点数 问题,确认 46 喔 5 数漏 发 5 帐单 缴费 13075 元 应道 13075 * ...,"[0.9991862, 0.9998606, 0.99667263, 0.99968255,..."
3,行销 活动 问题,未 收到 活动 点数 问题,问外 手机号码 卡通 缴 电费 没 回馈 谢谢,"[0.99937433, 0.9999896, 0.99837106, 0.9813643,..."
4,行销 活动 问题,未 收到 活动 点数 问题,hi 意 行销 条款 勾 绿色 勾勾 2 姓名 林 * 炜 身份证 f12 ...,"[0.9992154, 0.999914, 0.9928203, 0.99219674, 0..."


# create mutilable data

In [137]:
df = pd.read_csv(r"C:\Users\dupho\1_quinn/1Thesis/main_dataset/01trainingdata/raw_file.csv", index_col = 'id')

In [138]:
df

,label,detail_label,text
id,,,
0,行销 活动 问题,未 收到 活动 回馈 金 问题,2020 7 6 午 5 38 左右 绑定 兆丰 银行 开 启动 储值功...
1,行销 活动 问题,未 收到 活动 点数 问题,刷 屈臣氏 480 元数 回馈
2,行销 活动 问题,未 收到 活动 点数 问题,确认 46 喔 5 数漏 发 5 帐单 缴费 13075 元 应道 13075 * ...
3,行销 活动 问题,未 收到 活动 点数 问题,问外 手机号码 卡通 缴 电费 没 回馈 谢谢
4,行销 活动 问题,未 收到 活动 点数 问题,hi 意 行销 条款 勾 绿色 勾勾 2 姓名 林 * 炜 身份证 f12 ...
...,...,...,...
6949,注册 一卡通 帐户 问题,未成年 注册 问题,类 line pay money 注册 问题 未 成 注册 问题 名...
6950,储值 问题,连结 银行 问题,进 线 台湾银行 结 失败
6951,注册 一卡通 帐户 问题,金融 验证 问题,目前 遇 问题 注册 阶段 收 绑定 帐户 法 进行 员 注册 收 简讯 帐户 line ...


In [139]:
df.label.value_counts()

帐号 问题           2779
行销 活动 问题        1433
注册 一卡通 帐户 问题    1203
生活 缴费            402
储值 问题            344
收 付款 问题          334
转帐 问题            140
电票 连电支 问题        111
其他 问题             78
提领 问题             72
乘车 码 问题           51
Name: label, dtype: int64

In [140]:
# def get_col(row):
#     labels = ['帐号 问题','行销 活动 问题','注册 一卡通 帐户 问题','生活 缴费','储值 问题','收 付款 问题', '转帐 问题', '电票 连电支 问题', '其他 问题','提领 问题', '乘车 码 问题']
#     for i in range(6947): 
#         for j in labels:
#             if df.iloc[i] == j:
#                 return 1
#             else:
#                 return 0
        
    

In [145]:
labels = ['帐号 问题','行销 活动 问题','注册 一卡通 帐户 问题','生活 缴费','储值 问题','收 付款 问题', '转帐 问题', '电票 连电支 问题', '其他 问题','提领 问题', '乘车 码 问题']
for i in labels:
    df[i] = [(1 if i in element else 0) for element in df['label']]

In [146]:
df

,label,detail_label,text,帐号 问题,行销 活动 问题,注册 一卡通 帐户 问题,生活 缴费,储值 问题,收 付款 问题,转帐 问题,电票 连电支 问题,其他 问题,提领 问题,乘车 码 问题
id,,,,,,,,,,,,,,
0,行销 活动 问题,未 收到 活动 回馈 金 问题,2020 7 6 午 5 38 左右 绑定 兆丰 银行 开 启动 储值功...,0,1,0,0,0,0,0,0,0,0,0
1,行销 活动 问题,未 收到 活动 点数 问题,刷 屈臣氏 480 元数 回馈,0,1,0,0,0,0,0,0,0,0,0
2,行销 活动 问题,未 收到 活动 点数 问题,确认 46 喔 5 数漏 发 5 帐单 缴费 13075 元 应道 13075 * ...,0,1,0,0,0,0,0,0,0,0,0
3,行销 活动 问题,未 收到 活动 点数 问题,问外 手机号码 卡通 缴 电费 没 回馈 谢谢,0,1,0,0,0,0,0,0,0,0,0
4,行销 活动 问题,未 收到 活动 点数 问题,hi 意 行销 条款 勾 绿色 勾勾 2 姓名 林 * 炜 身份证 f12 ...,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6949,注册 一卡通 帐户 问题,未成年 注册 问题,类 line pay money 注册 问题 未 成 注册 问题 名...,0,0,1,0,0,0,0,0,0,0,0
6950,储值 问题,连结 银行 问题,进 线 台湾银行 结 失败,0,0,0,0,1,0,0,0,0,0,0
6951,注册 一卡通 帐户 问题,金融 验证 问题,目前 遇 问题 注册 阶段 收 绑定 帐户 法 进行 员 注册 收 简讯 帐户 line ...,0,0,1,0,0,0,0,0,0,0,0


In [147]:
df['行销 活动 问题'].value_counts()

0    5514
1    1433
Name: 行销 活动 问题, dtype: int64

In [148]:
df.label.value_counts()

帐号 问题           2779
行销 活动 问题        1433
注册 一卡通 帐户 问题    1203
生活 缴费            402
储值 问题            344
收 付款 问题          334
转帐 问题            140
电票 连电支 问题        111
其他 问题             78
提领 问题             72
乘车 码 问题           51
Name: label, dtype: int64

In [149]:
df.to_csv('clean_sim_multilabel.csv')